In [13]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 500)
import math
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import warnings
warnings.filterwarnings(action="ignore")

In [38]:
# Read df_crawled_routes
df_crawled_routes = pd.read_excel("flight_data_t_plus_6.xlsx").drop_duplicates()

# Read df_routes
df_routes = pd.read_excel("flight_routes.xlsx")
df_routes.columns = df_routes.columns.str.lower().str.replace(" ", "_")

# Read the airport data
df_airport_data = pd.read_excel("airport_data.xlsx")
# Add the latitude and longitude to the dataset
df_airport_data[["latitude", "longitude"]] = df_airport_data["Location"].str.split(",", 1, expand=True)

In [39]:
# Create a function to calculate the distance between two airports based on their latitude and longitude
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on the Earth's surface
    given their latitude and longitude coordinates.
    """
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    radius = 6371  # Radius of the Earth in kilometers
    distance = radius * c

    return round(distance, 2)

In [40]:
# Add the latitude and longitude fields of the *origin* airport
df_crawled_routes = pd.merge(
    left=df_crawled_routes,
    right=df_airport_data[["IataCode", "latitude", "longitude"]].drop_duplicates("IataCode"),
    left_on="origin_city_id",
    right_on="IataCode",
    how="left"
).drop(["IataCode"], axis=1)

# Add the latitude and longitude fields of the *destination* airport
df_crawled_routes = pd.merge(
    left=df_crawled_routes,
    right=df_airport_data[["IataCode", "latitude", "longitude"]].drop_duplicates("IataCode"),
    left_on="arrival_city_id",
    right_on="IataCode",
    how="left",
    suffixes=["_origin_city", "_arrival_city"]
).drop(["IataCode"], axis=1)

# Change the lat/long columns to type float
df_crawled_routes[["latitude_origin_city", "longitude_origin_city", "latitude_arrival_city", "longitude_arrival_city"]] = \
    df_crawled_routes[["latitude_origin_city", "longitude_origin_city", "latitude_arrival_city", "longitude_arrival_city"]].apply(lambda x: pd.to_numeric(x))

# Add the route competition status from df_routes
df_crawled_routes = pd.merge(
    left=df_crawled_routes,
    right=df_routes,
    left_on=["origin_city_search_term", "arrival_city_search_term"],
    right_on=["departure_city", "arrival_city"],
    how="left"
)

# Change all instances of Wizz Air {x} to just Wizz Air and replace any occurrence of a dot or space character in the "competitor" with an underscore. Finally, change all competitor names to lower case
df_crawled_routes["competitor"] = df_crawled_routes["competitor"].apply(lambda x: "Wizz" if "Wizz" in x else x).apply(lambda x: x.replace(".", "_").replace(" ", "_").lower())

In [41]:
# Add the distance field
df_crawled_routes["flight_distance_km"] = df_crawled_routes.\
    apply(lambda x: 
        haversine_distance(
            lat1=x["latitude_origin_city"],
            lon1=x["longitude_origin_city"],
            lat2=x["latitude_arrival_city"],
            lon2=x["longitude_arrival_city"]
        ), axis=1
    )

# Filter for crawled_routes with stop_count = 0
df_crawled_routes_stop_count_0 = df_crawled_routes[df_crawled_routes["stop_count"]==0]

In [42]:
# Choose the relevant columns from the data frame and melt it
relevant_cols = [
    "origin_city_search_term", "arrival_city_search_term", "route_competition_status",
    "origin_city_id", "latitude_origin_city", "longitude_origin_city", "origin_airport_name", "origin_airport_display_code",
    "arrival_city_id", "latitude_arrival_city", "longitude_arrival_city",  "arrival_airport_name", "arrival_airport_display_code",
    "stop_count", "competitor", "price_eur", "flight_duration", "flight_distance_km", "crawling_date"
]

# Sort the dataset by "crawling_date", "origin_city_search_term", "arrival_city_search_term", "competitor", "price_eur
sort_cols = ["crawling_date", "origin_city_search_term", "arrival_city_search_term", "competitor", "price_eur", "flight_duration"]
df_crawled_routes_stop_count_0 = df_crawled_routes_stop_count_0[relevant_cols].sort_values(sort_cols).drop_duplicates().reset_index(drop=True)

# Pick the cheapest price_eur in each partition
# We don't include the airports, so we can get one price_eur per competitor
partition_1_cols = [
    "crawling_date", "origin_city_search_term", "origin_city_id", "latitude_origin_city", "longitude_origin_city",
    "arrival_city_search_term", "arrival_city_id", "latitude_arrival_city", "longitude_arrival_city",
    "route_competition_status", "competitor", "stop_count"
]
df_crawled_routes_stop_count_0["min_price_flag"] = df_crawled_routes_stop_count_0.groupby(partition_1_cols, as_index=False)["price_eur"].rank(method="first", ascending=True)
df_min_price_per_comp_route = df_crawled_routes_stop_count_0[df_crawled_routes_stop_count_0["min_price_flag"] == 1]

# Inner join df_min_price_per_comp_route_stop_count and df_final to eliminate the more expensive flights
df_final = pd.merge(
    left=df_min_price_per_comp_route,
    right=df_crawled_routes_stop_count_0[["origin_city_search_term", "arrival_city_search_term"]].drop_duplicates(),
    on=["origin_city_search_term", "arrival_city_search_term"],
    how="inner"
)

df_final

,origin_city_search_term,arrival_city_search_term,route_competition_status,origin_city_id,latitude_origin_city,longitude_origin_city,origin_airport_name,origin_airport_display_code,arrival_city_id,latitude_arrival_city,longitude_arrival_city,arrival_airport_name,arrival_airport_display_code,stop_count,competitor,price_eur,flight_duration,flight_distance_km,crawling_date,min_price_flag
0,Aarhus,London (GB),Monopoly not always,AAR,56.310278,10.618056,Aarhus,AAR,LOND,51.504117,-0.094347,London Stansted,STN,0,ryanair,84.45,105,880.42,2023-12-18,1.0
1,Agadir,Frankfurt,1 comp No Wizz,AGA,30.325619,-9.412708,Agadir,AGA,FRAN,50.117272,8.644397,Frankfurt am Main,FRA,0,tuifly,180.00,255,2666.72,2023-12-18,1.0
2,Agadir,London (GB),2+ comps inc Wizz,AGA,30.325619,-9.412708,Agadir,AGA,LOND,51.504117,-0.094347,London Gatwick,LGW,0,easyjet,67.76,225,2476.97,2023-12-18,1.0
3,Agadir,London (GB),2+ comps inc Wizz,AGA,30.325619,-9.412708,Agadir,AGA,LOND,51.504117,-0.094347,London Gatwick,LGW,0,tui,256.67,225,2476.97,2023-12-18,1.0
4,Agadir,Manchester (GB),2+ comps exc Wizz,AGA,30.325619,-9.412708,Agadir,AGA,MAN,53.358812,-2.272687,Manchester,MAN,0,easyjet,59.56,235,2625.46,2023-12-18,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388,Zaragoza,London (GB),Monopoly always,ZAZ,41.663423,-1.011086,Zaragoza,ZAZ,LOND,51.504117,-0.094347,London Stansted,STN,0,ryanair,33.99,130,1096.45,2023-12-18,1.0
3389,Zaragoza,Marrakech,Monopoly always,ZAZ,41.663423,-1.011086,Zaragoza,ZAZ,RAK,31.608333,-8.038333,Marrakech Menara,RAK,0,ryanair,50.14,125,1280.78,2023-12-18,1.0
3390,Zaragoza,Milan,Monopoly always,ZAZ,41.663423,-1.011086,Zaragoza,ZAZ,MILA,45.468394,9.173017,Milan Bergamo,BGY,0,ryanair,28.99,110,922.24,2023-12-18,1.0
3391,Zaragoza,Palma de Mallorca,2+ comps exc Wizz,ZAZ,41.663423,-1.011086,Zaragoza,ZAZ,PMI,18.729167,-91.656944,Palma - Majorca,PMI,0,vueling_airlines,25.99,65,8689.08,2023-12-18,1.0


In [43]:
# Change the data frame so that each competitor's price is shown as one column
pivot_table_index_cols = [
    "crawling_date", "origin_city_search_term", "origin_city_id", "latitude_origin_city", "longitude_origin_city",
    "arrival_city_search_term", "arrival_city_id", "latitude_arrival_city", "longitude_arrival_city", "route_competition_status", "flight_distance_km", "stop_count"
] 
df_final_pivot = pd.pivot_table(
    data=df_final,
    values="price_eur",
    index=pivot_table_index_cols,
    columns=["competitor"]
).reset_index()

# Display Ryan Air and Wizz as the first two competitors
all_df_final_pivot_cols = df_final_pivot.columns.tolist()
first_cols = pivot_table_index_cols + ["ryanair", "wizz"]
exclusion_list = list(set(all_df_final_pivot_cols) - set(first_cols))
rearranged_col_list = first_cols + exclusion_list 
df_final_pivot = df_final_pivot[rearranged_col_list]
# Remove the name "competitor" from the index
df_final_pivot.rename_axis(None, inplace=True, axis=1)

# Display the final table
df_final_pivot

,crawling_date,origin_city_search_term,origin_city_id,latitude_origin_city,longitude_origin_city,arrival_city_search_term,arrival_city_id,latitude_arrival_city,longitude_arrival_city,route_competition_status,...,singapore_airlines,transavia,tuifly,easyjet_europe,aer_lingus,brussels_airlines,tuifly_be,iberia,air_arabia_maroc,ethiopian_airlines
0,2023-12-18,Aarhus,AAR,56.310278,10.618056,London (GB),LOND,51.504117,-0.094347,Monopoly not always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-12-18,Agadir,AGA,30.325619,-9.412708,Frankfurt,FRAN,50.117272,8.644397,1 comp No Wizz,...,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-12-18,Agadir,AGA,30.325619,-9.412708,London (GB),LOND,51.504117,-0.094347,2+ comps inc Wizz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-12-18,Agadir,AGA,30.325619,-9.412708,Manchester (GB),MAN,53.358812,-2.272687,2+ comps exc Wizz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-18,Agadir,AGA,30.325619,-9.412708,Nantes,NTE,47.158333,-1.609722,1 comp No Wizz,...,NaN,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,2023-12-18,Zaragoza,ZAZ,41.663423,-1.011086,London (GB),LOND,51.504117,-0.094347,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1886,2023-12-18,Zaragoza,ZAZ,41.663423,-1.011086,Marrakech,RAK,31.608333,-8.038333,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1887,2023-12-18,Zaragoza,ZAZ,41.663423,-1.011086,Milan,MILA,45.468394,9.173017,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1888,2023-12-18,Zaragoza,ZAZ,41.663423,-1.011086,Palma de Mallorca,PMI,18.729167,-91.656944,2+ comps exc Wizz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Append the rest of the routes that did not have direct flights to df_final_pivot

# First create a data frame containing the indirect routes
df_indirect_routes = pd.merge(
    left=df_crawled_routes[pivot_table_index_cols[0:-1]].drop_duplicates().reset_index(drop=True),
    right=df_crawled_routes_stop_count_0[pivot_table_index_cols].drop_duplicates().reset_index(drop=True),
    on=pivot_table_index_cols[0:-1],
    how="left"
)
df_indirect_routes = df_indirect_routes[df_indirect_routes["stop_count"].isnull()].reset_index(drop=True)
df_indirect_routes.fillna(value="> 0 stops", inplace=True)

# Next, create df_final_pivot 
df_final_pivot_full = pd.concat(
    [df_final_pivot, df_indirect_routes], axis=0
)

# Convert stop_count to string
df_final_pivot_full["stop_count"] = df_final_pivot_full["stop_count"].apply(lambda x: str(x))

# Add a partition field to allow for easier filtering
def partition_func(df, ryanair_col, stop_count_col):
    if ~np.isnan(df[ryanair_col]) and df[stop_count_col] == "0":
        return "partition_1"
    elif np.isnan(df[ryanair_col]) and df[stop_count_col] == "0":
        return "partition_2"
    elif df[stop_count_col] == "> 0 stops":
        return "partition_3"

df_final_pivot_full["data_partition"] = df_final_pivot_full.apply(partition_func, axis=1, ryanair_col="ryanair", stop_count_col="stop_count")

In [37]:
# Upload the data to BigQuery
credentials = service_account.Credentials.from_service_account_file(
    filename=os.path.expanduser("~") + "/bq_credentials.json", scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
client = bigquery.Client(project="web-scraping-371310", credentials=credentials)
job_config_raw = bigquery.LoadJobConfig(
    schema=[

    ]
)
job_config_raw.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

job_config_pivot = bigquery.LoadJobConfig()
job_config_pivot.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Upload the raw data
client.load_table_from_dataframe(
    dataframe=df_crawled_routes,
    destination="web-scraping-371310.crawled_datasets.benoit_flight_route_data_raw",
    job_config=job_config_pivot
).result()

# Upload the pivoted table
client.load_table_from_dataframe(
    dataframe=df_final_pivot_full,
    destination="web-scraping-371310.crawled_datasets.benoit_flight_route_data_pivoted",
    job_config=job_config_pivot
).result()


LoadJob<project=web-scraping-371310, location=EU, id=51e3d12a-06c2-43a6-8214-d6c4da72a4ef>